# COMP30027 Machine Learning, 2018 Sem 1
-----
## Project 1: What is labelled data worth to Naive Bayes?
-----
###### Student Name: Nico Dinata (770318)
###### Python version: 3.6

In [484]:
import pandas as pd                     # mostly for pd.DataFrame.iloc[]
import numpy as np                      # solely for np.random
import math
from collections import defaultdict     # the backbone of this project

In [485]:
# This function should open a data file in csv, and transform it into a usable format 
def preprocess(filename):
    df = pd.read_csv(filename, header=None)
    
    # Handle missing values represented by a "?", by removing instances
    # where it's present
    for i in range(len(df.columns)):
        df = df[(df[i].map(str) != "?")]
    
    # Reset index after removing some instances
    df = df.reset_index(drop=True)
    
    # Separate dataset into table of attributes and that of the classes
    list_attribute = df.iloc[:, :(len(df.columns)-1)]
    list_class = df.iloc[:, len(df.columns)-1]

    return (list_attribute, list_class)


test = preprocess('./test.csv')
breast_cancer = preprocess('./data/breast-cancer.csv')
car = preprocess('./data/car.csv')
hypothyroid = preprocess('./data/hypothyroid.csv')
mushroom = preprocess('./data/mushroom.csv')

In [486]:
# This function should build a supervised NB model
def train_supervised(dataset):
    list_attribute = dataset[0]
    list_class     = dataset[1]
    
    # --------------------------------------------------------------------- #
    # Prior probability
    
    # Count number of instances of each class
    classes = defaultdict(int)
    for c in list_class:
        classes[c] += 1
    
    # The total number of instances
    total_class_count = sum(classes.values())
    
    # Create a dictionary containing the probabilistic model of the classes
    # (prior probability)
    prior_probability_model = defaultdict(int)
    for k in classes.keys():
        prior_probability_model[k] = (classes[k]/total_class_count)


    # --------------------------------------------------------------------- #
    # Posterior probability
        
    # Create a dictionary of dictionaries of dictionaries (yikes) as
    # representation of instances of data (default 1 for Laplace smoothing)
    instances = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: 1)))
    
    # Set to keep track of the distinct attribute values of each attribute
    for j in range(len(list_attribute.columns)):
        instances[j]["attribute_values"] = set()
        
    # Count number of instances of a particular attribute value given a
    # particular class, while also adding that attribute value to the set
    for i in range(len(list_class)):
        for j in range(len(list_attribute.columns)):
            instances[j][list_class[i]][list_attribute.iloc[i,j]] += 1
            instances[j]["attribute_values"].add(list_attribute.iloc[i,j])
    
    # Add 1 to the instances of attribute values the model has never seen before
    # for a particular class (Laplace smoothing)
    for attrib,v in instances.items():
        for attrib_instance in instances[attrib]["attribute_values"]:
            for c,v2 in v.items():
                if (c != "attribute_values" and instances[attrib][c][attrib_instance] == 0):
                    instances[attrib][c][attrib_instance] = 1
    
    # Create the probabilistic model of the attribute value instances
    posterior_probability_model = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))

    for attrib,v in instances.items():
        for c,v2 in v.items():
            if (c != "attribute_values"):
                for attrib_value,v3 in v2.items():
                    posterior_probability_model[c][attrib][attrib_value] = (v3/sum(v2.values()))
    
    return (prior_probability_model, posterior_probability_model)


model_test = train_supervised(test)
model_bc = train_supervised(breast_cancer)
model_c = train_supervised(car)
model_h = train_supervised(hypothyroid)
model_m = train_supervised(mushroom)

In [487]:
# This function should predict the class for a set of instances, based on a trained model 
def predict_supervised(model, dataset):
    list_attribute = dataset[0]
    prior_probability_model = model[0]
    posterior_probability_model = model[1]
    predictions = []
    
    # For each data instance, generate a score for each class and append the class with
    # the highest score to the list of predictions
    for i in range(len(list_attribute)):
        scores = {}
        for c,v in posterior_probability_model.items():
            score = math.log(prior_probability_model[c])
            for j in range(len(list_attribute.columns)):
                score += math.log(posterior_probability_model[c][j][list_attribute.iloc[i,j]])
            scores[c] = score

        v = list(scores.values())
        k = list(scores.keys())
        predictions.append(k[v.index(max(v))])
    
    return predictions


predictions_test = predict_supervised(model_test, test)
predictions_bc = predict_supervised(model_bc, breast_cancer)
predictions_c = predict_supervised(model_c, car)
predictions_h = predict_supervised(model_h, hypothyroid)
predictions_m = predict_supervised(model_m, mushroom)

In [489]:
# This function should evaluate a set of predictions, in a supervised context 
def evaluate_supervised(predictions, dataset):
    list_class = dataset[1]
    
    # Count the number of correct/matching predictions
    correct = 0
    for i in range(len(list_class)):
        correct += (predictions[i] == list_class[i])
    
    # Accuracy percentage
    result = 100 * correct/len(list_class)
    return round(result,3)


print("ACCURACY")
print("--------")
print("{} {}{}".format("Test:", evaluate_supervised(predictions_test, test), "%"))
print("{} {}{}".format("Breast cancer:", evaluate_supervised(predictions_bc, breast_cancer), "%"))
print("{} {}{}".format("Car:", evaluate_supervised(predictions_c, car), "%"))
print("{} {}{}".format("Hypothyroid:", evaluate_supervised(predictions_h, hypothyroid), "%"))
print("{} {}{}".format("Mushroom:", evaluate_supervised(predictions_m, mushroom), "%"))

# ACCURACY
# --------
# Test: 100.0%
# Breast cancer: 76.895%
# Car: 87.153%
# Hypothyroid: 95.146%
# Mushroom: 97.697%

ACCURACY
--------
Test: 100.0%
Breast cancer: 76.895%
Car: 87.153%
Hypothyroid: 95.146%
Mushroom: 97.697%


In [490]:
# This function should build an unsupervised NB model 
def train_unsupervised(dataset, classes, class_distribution):
    list_class = []
    
    # Assign random class distribution if not given one
    if (not class_distribution):    
        for i in range(len(dataset)):
            class_distribution = {}
            rand_dis = np.random.random(len(classes))
            rand_dis /= rand_dis.sum()
        
            for j in range(len(classes)):
                class_distribution[classes[j]] = rand_dis[j]
            list_class.append(class_distribution)
    else:
        list_class = class_distribution
        

    # --------------------------------------------------------------------- #
    # Prior probability
    
    # Count the fractional value of each class
    class_dist = defaultdict(int)
    for c_dict in list_class:
        for c,v in c_dict.items():
            class_dist[c] += v
    
    # The total number of instances
    total_class_count = sum(class_dist.values())
    
    # Create a dictionary containing the probabilistic model of the classes
    # (prior probability)
    prior_probability_model = defaultdict(int)
    for k in class_dist.keys():
        prior_probability_model[k] = (class_dist[k]/total_class_count)
    

    # --------------------------------------------------------------------- #
    # Posterior probability
    
    # Create a dictionary of dictionaries of dictionaries as representation
    # of instances of data
    instances = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))
        
    # Count number of instances of a particular attribute value given a
    # particular class
    for i in range(len(list_class)):
        for j in range(len(dataset.columns)):
            for c,v in list_class[i].items():
                instances[j][c][dataset.iloc[i,j]] += v
    
    # Create the probabilistic model of the attribute value instances
    posterior_probability_model = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))

    for attrib,v in instances.items():
        for c,v2 in v.items():
            for attrib_value,v3 in v2.items():
                posterior_probability_model[c][attrib][attrib_value] = (v3/sum(v2.values()))
    
    return (prior_probability_model, posterior_probability_model)


In [494]:
# This function should predict the class distribution for a set of instances, based on a trained model
def predict_unsupervised(model, dataset):
    prior_probability_model = model[0]
    posterior_probability_model = model[1]
    new_class_distribution = []
    
    # For each data instance, generate a score for each class to become a more "reliable"
    # class distribution in the next iteration
    for i in range(len(dataset)):
        scores = {}
        for c,v in posterior_probability_model.items():
#             score = math.log(prior_probability_model[c])
            score = prior_probability_model[c]
            for j in range(len(dataset.columns)):
#                 score += math.log(posterior_probability_model[c][j][dataset.iloc[i,j]])
                score *= posterior_probability_model[c][j][dataset.iloc[i,j]]
#             scores[c] = math.exp(score)
            scores[c] = score
        
        # Normalise the scores
        class_sum = sum(scores.values())
        for c,v in scores.items():
            scores[c] = v/class_sum
        
        # Add the new class distribution
        new_class_distribution.append(scores)
    
    return new_class_distribution


In [495]:
# Iterate the process of training and "predicting", to (hopefully) improve the probability estimates 
def iterate(dataset, classes, init_class_distribution):
    predictions = []
    model = train_unsupervised(dataset, classes, init_class_distribution)
    
    # Iterate for 10 times
    for i in range(10):
        new_class_distribution = predict_unsupervised(model, dataset)
        model = train_unsupervised(dataset, classes, new_class_distribution)
    
    # Choose the most probable class as the "predicted" class
    for i in range(len(dataset)):
        v = list(new_class_distribution[i].values())
        k = list(new_class_distribution[i].keys())
        predictions.append(k[v.index(max(v))])
    
    return predictions

    
# predictions_test = iterate(test[0], ["flu", "cold"], [])
predictions_bc = iterate(breast_cancer[0], ["no-recurrence-events", "recurrence-events"], [])
predictions_c = iterate(car[0], ["acc", "unacc", "good", "vgood"], [])
predictions_h = iterate(hypothyroid[0], ["hypothyroid", "negative"], [])
predictions_m = iterate(mushroom[0], ["e", "p"], [])

In [496]:
# This function should evaluate a set of predictions, in an unsupervised manner
def evaluate_unsupervised(predictions, dataset):
    list_class = dataset[1]
    
    # Count the number of correct/matching predictions
    correct = 0
    for i in range(len(list_class)):
        correct += (predictions[i] == list_class[i])
    
    # Accuracy percentage
    result = 100 * correct/len(list_class)
    return round(result,3)


print("ACCURACY")
print("--------")
# print("{} {}{}".format("Test:", evaluate_unsupervised(predictions_test, test), "%"))
print("{} {}{}".format("Breast cancer:", evaluate_unsupervised(predictions_bc, breast_cancer), "%"))
print("{} {}{}".format("Car:", evaluate_unsupervised(predictions_c, car), "%"))
print("{} {}{}".format("Hypothyroid:", evaluate_unsupervised(predictions_h, hypothyroid), "%"))
print("{} {}{}".format("Mushroom:", evaluate_unsupervised(predictions_m, mushroom), "%"))

ACCURACY
--------
Breast cancer: 37.906%
Car: 32.986%
Hypothyroid: 87.249%
Mushroom: 24.592%


#### Question 1

> Since we’re starting off with random guesses, it might be surprising that the unsupervised NB works at all. Explain what characteristics of the data cause it to work pretty well (say, within 10% Accuracy of the supervised NB) most of the time; also, explain why it utterly fails sometimes.

After several iterations, it is observed that the learner consistently performs poorly on `car.csv`, while “failing” only sometimes on the other datasets. A notable characteristic of this dataset is its four classes, as opposed to two of the others. Firstly, we should consider that after several iterations to improve the probability estimates, the class distributions generally tend to converge to a particular class, forming a “reliable” prediction of the “correct” class.
<br>

Taking this into account, because `car.csv` has twice the classes of all the other datasets, it may require more iterations to be able to reliably reach a point where only one of its classes obtain a probability much higher than the rest of the classes (e.g. 0.97, 0.1, 0.1, 0.1); this leads to a somewhat “even” spread of probabilities among all the classes. On the flipside, having only two classes seems to expedite the process of reaching convergence, although there is an exception in `mushroom.csv`. It seems to produce a rather low success rate, a cause for which may be due to its large size (in terms of both instances and attributes).

#### Question 2

> When evaluating supervised NB across the four different datasets, you will observe some variation in effectiveness (e.g. Accuracy). Explain what causes this variation. Describe and explain any particularly suprising results.

One of the potential factors to the variation in accuracy could be the size of the data (number of instances); the higher the number of instances, the higher the chances of unique, previously-unseen attribute value combinations to appear, and thus the better the quality of the model constructed. It’s worth noting, though, that this is not a guarantee, an observation of which is detailed down below. The fact that Laplace smoothing (used in the supervised context of this project) tends to perform better with larger number of instances (doesn’t alter the probabilities too drastically) also plays a part, giving the overall effect of the learner performing better on larger datasets.
<br>

A particularly interesting result is the fact that the difference in size between `car.csv` and `hypothyroid.csv` is significantly smaller than that between `hypothyroid.csv` and `mushroom.csv`, and yet the leap in accuracy is much higher between `car.csv` and `hypothyroid.csv`. A potential reason is the different ways the attribute value combinations appear on each dataset, relating to the point mentioned above about them being probabilistic. It may also suggest that there is a “peak” number of instances, beyond which it gives diminishing returns in terms of accuracy.